# RAG系统基础:RAG系统基础：从零开始构建检索增强生成系统

RAG 系统解决的问题： **模型训练知识截至日期**和**领域专业知识的缺乏**

例如：在企业中询问 我们公司2025年Q3季度的销售数据是多少？（公司内部文件）

# RAG系统的组成

一个完整的RAG系统由三个核心组件构成：

索引阶段 Indexing -> 检索阶段 Retrieval -> 生成阶段 Generation

## Indexing

将文档转换为可搜索的向量格式

核心步骤：
    1. 文档加载 Document Loading
    2. 文本分块 Text Splitting
    3. 向量化 Vectorization
    4. 存储 Storage

## Retrieval

根据用户查询找到最相关的文档片段

核心步骤：
    1. 查询向量化
    2. 相似度计算
    3. 结果排序

## Generation

基于检索到的信息生成准确答案

核心步骤：
    1. 构建提示词
    2. LLM生成
    3. 答案返回

# 配置环境，导入核心包

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("DASHSCOPE_API_KEY")

from langchain_qwq import ChatQwen
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

# 索引 Indexing - 构建知识库
核心步骤：

1. 文档加载 Document Loading
2. 文本分块 Text Splitting
3. 向量化 Vectorization
4. 存储 Storage

## 步骤1：文档加载

In [ ]:
# 步骤1：首先需要加载文档。LangChain支持多种文档源
from langchain_community.document_loaders import(
    WebBaseLoader,      # 网页
    PyPDFLoader,        # PDF
    TextLoader,         # 文本文件
    DirectoryLoader,    # 目录
    CSVLoader,          # CSV
)

# Exa: Loading Webpage
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

print(f"加载了 {len(docs)} 个文档")
print(f"第一个文档长度： {(len(docs[0].page_content))} 字符")

## 步骤2：文本分块

为什么需要分块？
1. LLM上下文限制：大多数LLM有最大token限制（如GPT-4的8K/32K）
2. 提高检索精度：小块文本更容易匹配特定查询
3. 降低成本：只检索和处理相关部分

不同分块策略
```python
# 字符分块（最基础）
CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Token分块（精确控制token数量）
from langchain.text_splitter import TokenTextSplitter
TokenTextSplitter(chunk_size=256, chunk_overlap=50)

# 语义分块（基于语义相似度）
from langchain_experimental.text_splitter import SemanticChunker
SemanticChunker(OpenAIEmbeddings())

# 递归分块（智能识别段落）
RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]  # 优先按段落分割
)
```

In [ ]:
# 创建文本分块器
# 递归分块（智能识别段落）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,            # 每块最大字符数 推荐值 500-1500，太小 -> 上下文不足，太大 -> 匹配不精准
    chunk_overlap = 200,          # 块之间的重叠 10-20% 大小，保持上下文连贯性
    length_function = len,        # 长度计算函数
    is_separator_regex = False,
    separators=["\n\n", "\n", " ", ""]  # 分隔符：优先按段落分割，段落 > 行 > 词
)

# 分块文档
splits = text_splitter.split_documents(docs)

print(f"原始文档: {len(docs)} 个")
print(f"分块后: {len(splits)} 个")
print(f"\n第一个分块示例:\n{splits[0].page_content[:30]}...")

## 步骤3：向量化
将文本转换为数学向量，这是语义搜索的核心

In [ ]:
# 创建嵌入模型
embeddings_model = DashScopeEmbeddings(
    model="text-embedding-v4",
)

# Exa: 向量化一段文字
text = "RAG是一种强大的AI技术"
new_vector_text = embeddings_model.embed_query(text)

print(f"文本: {text}")
print(f"向量维度: {len(new_vector_text)}")
print(f"向量前5个值: {new_vector_text[:5]}")

## 步骤4：存储

向量数据库选择：

| **数据库**  | **类型** | **优势**      | **适用场景**   |
|----------|--------|-------------|------------|
| Chroma   | 嵌入式    | 简单易用，无需额外服务 | 开发、小规模应用 ⭐ |
| Pinecone | 云服务    | 高性能，托管服务    | 生产环境       |
| Weaviate | 自建     | 功能丰富，开源     | 大规模部署      |
| FAISS    | 库      | 速度快，Meta开源  | 研究和原型      |

In [ ]:
# 创建向量数据库
vectorstore = Chroma.from_documents(
    documents=splits,           # 分块后的文档
    embedding=embeddings_model,       # 嵌入模型
    persist_directory="./chroma_db"  # 持久化目录
)

print(f"✅ 向量数据库创建成功！")
print(f"   存储了 {len(splits)} 个文档块")
print(f"   持久化路径: ./chroma_db_")

## 完整流程 + 封装

In [4]:
def create_vectorstore(url: str, chunk_size: int = 1000):
    """创建向量数据库"""

    # 1. 加载文档
    loader = WebBaseLoader(url)
    docs = loader.load()

    # 2. 分块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size * 0.1)
    )
    splits = text_splitter.split_documents(docs)

    # 3. 向量化并存储
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=DashScopeEmbeddings(model="text-embedding-v4"),
        persist_directory=f"./chroma_db_{url.split('/')[-1]}"
    )

    return vectorstore

# 使用
vectorstore = create_vectorstore(
    "https://lilianweng.github.io/posts/2023-06-23-agent/"
)

# 检索 Retrieval -找到相关信息

In [9]:
# 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 相似度搜索
    search_kwargs={"k": 3}     # 返回前3个结果
)

# 执行检索
query = "什么是Agent？"
docs = retriever.invoke(query)

# 查看结果
for i, doc in enumerate(docs, 1):
    print(f"\n📄 结果 {i}:")
    print(f"内容: {doc.page_content[:200]}...")
    print(f"来源: {doc.metadata.get('source', 'N/A')}")


📄 结果 1:
内容: Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:...
来源: https://lilianweng.github.io/posts/2023-06-23-agent/

📄 结果 2:
内容: Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:...
来源: https://lilianweng.github.io/posts/2023-06-23-agent/

📄 结果 3:
内容: Prompt LM with 100 most recent observations and to generate 3 most salient high-level questions given a set of observations/statements. Then ask LM to answer those questions.


Planning & Reacting: tr...
来源: https://lilianweng.github.io/posts/2023-06-23-agent/


# 生成 Generation -产生答案

In [43]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 创建LLM
llm = ChatQwen(
    model="qwen-plus",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0  # 0 = 更精确, 1 = 更有创意
)

# 定义 prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个AI助手。请根据以下背景信息回答用户的问题。\n\n背景信息：\n{context}"),
    ("human", "{query}"),
])


# 1. 定义格式化函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 2. 定义并行运行的步骤（既保留文档对象，又生成 prompt 需要的字符串）
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)

# 3. 定义生成答案的分支
qa_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) # 这里只把 context 转字符串给 prompt 用
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 合并链：先检索，然后把结果由 assign 加到字典里
final_chain = setup_and_retrieval.assign(answer=qa_chain)

# 5. 提问
query = "什么是Agent？它有什么能力？"
result = final_chain.invoke(query)

# 6. 结果现在是一部字典，包含所有信息
print("问题:", result["query"])
print("回答:", result["answer"])
print("来源:", result["context"]) # 这里就是 source_documents

回答: 什么是Agent？它有什么能力？
回答: **什么是 Agent？**

在人工智能领域，**Agent（智能代理）** 是一个能够感知环境、进行决策并采取行动以实现特定目标的自主实体。它可以是一个软件程序、机器人，或更广泛地指任何具备一定“智能”行为的系统。

当提到 **LLM-powered Agent（大语言模型驱动的智能体）** 时，它指的是以大型语言模型（如 GPT、通义千问等）为核心的大脑，结合外部工具和记忆系统，形成的一个**自主完成任务的智能系统**。

---

### **Agent 的核心能力**

一个典型的 LLM Agent 具备以下四大核心能力：

#### 1. **感知与理解（Perception & Understanding）**
- 能够接收用户指令、环境输入（如文本、文件、网页内容等）。
- 利用语言模型的强大语义理解能力，解析意图、提取关键信息。

> ✅ 示例：你告诉 Agent “帮我查一下今天北京的天气”，它能理解“今天”、“北京”、“天气”这些关键词及其关系。

---

#### 2. **规划与推理（Planning & Reasoning）**
- 可将复杂任务拆解为多个子任务，制定执行计划。
- 在执行过程中动态调整策略，处理异常或新情况。

> ✅ 示例：要“写一份关于新能源汽车市场的分析报告”，Agent 会自动规划为：
> - 搜索最新行业数据
> - 分析竞争格局
> - 整理趋势与风险
> - 组织成文

---

#### 3. **工具使用（Tool Use / Function Calling）**
- 能调用外部 API 或工具来弥补自身局限（如无法直接访问实时信息）。
- 常见工具包括：
  - 浏览器搜索（获取实时资讯）
  - 代码解释器（做数学计算、数据可视化）
  - 文件读写（保存结果）
  - 数据库查询
  - 邮件发送接口等

> ✅ 示例：查询汇率、运行 Python 脚本画图、下载网页内容。

---

#### 4. **记忆与学习（Memory Management）**
- **短期记忆**：上下文窗口内的对话历史（类似工作记忆）。
- **长期记忆**：通过向量数据库存储和检索过往经验、知识、用户偏好等，实现跨会话记忆。

> ✅ 示例：记住你上周说